In [1]:
import json

In [2]:
data_list = json.loads(open("gnpslibraryjson.json").read())

In [4]:
new_data_list = []
for element in data_list:
    if len(element["InChIKey_inchi"]) < 5 and len(element["InChIKey_smiles"]) < 5:
        continue
    new_data_list.append(element)

In [20]:
#Formatting
output_list = []
for element in new_data_list:
    inchi_key = element["InChIKey_inchi"]
    if len(element["InChIKey_smiles"]) > len(inchi_key):
        inchi_key = element["InChIKey_smiles"]
    
    output_dict = {}
    output_dict["GNPSID"] = element["spectrum_id"]
    output_dict["COMPOUND_NAME"] = element["Name"]
    output_dict["COMPOUND_INCHIKEY"] = inchi_key
    
    output_list.append(output_dict)
    
print("GNPS Library Entries with INCHIKEY", len(output_list))
open("gnpslibraryjson_filtered.json", "w").write(json.dumps(output_list, indent=4))

GNPS Library Entries with INCHIKEY 77252


17134708

In [12]:
discordant_list = [element for element in new_data_list \
                   if (len(element["InChIKey_inchi"]) == len(element["InChIKey_smiles"]) \
                       and element["InChIKey_inchi"].split("-")[0] != element["InChIKey_smiles"].split("-")[0])]

In [14]:
len(discordant_list)

10

In [15]:
discordant_list[0]

{'InChI': 'InChI=1S/C15H21N3O2/c1-15-7-8-17(3)13(15)18(4)12-6-5-10(9-11(12)15)20-14(19)16-2/h5-6,9,13H,7-8H2,1-4H3,(H,16,19)',
 'InChIKey_inchi': 'PIJVFDBKTWXHHD-UHFFFAOYSA-N',
 'InChIKey_smiles': 'MVAWJSIDNICKHF-UHFFFAOYSA-N',
 'Name': 'N-acetyl-5-hydroxytryptamine',
 'SMILES': 'OC1=CC=C(NC=C2CCNC(C)=O)C2=C1',
 'spectrum_id': 'CCMSLIB00000081266',
 'url': 'https://gnps.ucsd.edu/ProteoSAFe/gnpslibraryspectrum.jsp?SpectrumID=CCMSLIB00000081266'}

In [14]:
import requests
import json
import pandas as pd

In [3]:
r = requests.get("http://dorresteinappshub.ucsd.edu:5010/gnpslibraryfornpatlasjson")
gnps_list = r.json()

In [60]:
len(gnps_list)

77252

In [6]:
npatlas_list = json.load(open("data/npatlas.json", encoding='utf-8', errors='strict'), strict=False)

In [38]:
all_gnps_keys = set([element["COMPOUND_INCHIKEY"].split("-")[0] for element in gnps_list])
all_npatlas_keys = set([element["COMPOUND_INCHIKEY"].split("-")[0] for element in npatlas_list])
all_npatlas_name = set([element["COMPOUND_NAME"] for element in npatlas_list])

In [38]:
print(len(all_gnps_keys))
print(len(all_npatlas_keys))

10024
19350


In [40]:
len(all_gnps_keys.intersection(all_npatlas_keys))

522

In [28]:
def find_compound_name(compound_name, compound_df):
    filtered_df = compound_df.loc[compound_df['COMPOUND_NAME'] == compound_name]
    if len(filtered_df) > 0:
        return filtered_df.to_dict(orient="records")
    return None

def find_inchi_key(inchikey, compound_df):
    filtered_df = compound_df.loc[compound_df['COMPOUND_INCHIKEYBLOCK1'] == inchikey.split("-")[0]]
    if len(filtered_df) > 0:
        return filtered_df.to_dict(orient="records")
    return None

In [36]:
#Trying to determine if we can find discordant annotations
found_in_npatlas = []
npatlas_df = pd.DataFrame(npatlas_list)
npatlas_df['COMPOUND_INCHIKEYBLOCK1'] = npatlas_df.apply(lambda x: x["COMPOUND_INCHIKEY"].split("-")[0], axis=1)

for gnps_compound in gnps_list:
    gnps_compound_name = gnps_compound["COMPOUND_NAME"]
    gnps_inchikey = gnps_compound["COMPOUND_INCHIKEY"]
    np_atlas_compound_by_name = find_compound_name(gnps_compound_name, npatlas_df)
    np_atlas_compound_by_key = find_inchi_key(gnps_inchikey, npatlas_df)

    if np_atlas_compound_by_key != None:
        found_in_npatlas.append(gnps_compound)
        continue
    
    if np_atlas_compound_by_name != None:
        #print(gnps_compound_name, np_atlas_compound_by_name, np_atlas_compound_by_key)
        print(gnps_compound_name, gnps_inchikey, np_atlas_compound_by_key)


Abyssomicin J HEIHBBPLLFNNDR-ONVIIIJXSA-N None
Pyochelin VRTHPBIMOLQLGL-MXZFKARLSA-N None
Arylomycin A4 FWGUGBNIAQFVCD-VMUCWRCRSA-N None
Stenothricin A FKLMOAHXSWVYLL-ASLZBUMRSA-N None
Grisorixin NMHMUVNIGSQOII-UHFFFAOYSA-N None
Cyclosporin A PMATZTZNYRCHOR-IMVLJIQESA-N None
Chromomycin A3 ZYVSOIYQKUDENJ-UHFFFAOYSA-N None
Chromomycin A3 ZYVSOIYQKUDENJ-UHFFFAOYSA-N None
Chromomycin A3 ZYVSOIYQKUDENJ-UHFFFAOYSA-N None
Alanopine FIOHTMQGSFVHEZ-UHFFFAOYSA-N None
Alanopine FIOHTMQGSFVHEZ-UHFFFAOYSA-N None
Piperazimycin B WZHWJPRBJXEKMW-UHFFFAOYSA-N None
Dinactin CTZNINKRTKCWGU-UHFFFAOYSA-N None
Lipid A WGSFUVTXMVWHBP-JDEZGVNFSA-N None
Bisucaberin GTADQMQBQBOJIO-UHFFFAOYSA-N None


KeyboardInterrupt: 

In [50]:
#Trying to determine if we can upgrade structures

r = requests.get("http://dorresteinappshub.ucsd.edu:5010/gnpslibraryformattedjson")
all_gnps_without_structure_fiter = r.json()

In [61]:
count = 1
for gnps_compound in all_gnps_without_structure_fiter:
    gnps_compound_name = gnps_compound["COMPOUND_NAME"]
    gnps_inchikey = gnps_compound["COMPOUND_INCHIKEY"]
    gnps_inchikeyblock1 = gnps_inchikey.split("-")[0]
    
    if gnps_compound_name in all_npatlas_name:
        np_atlas_compound_by_name = find_compound_name(gnps_compound_name, npatlas_df)
        npatlas_inchikeyblock1 = np_atlas_compound_by_name[0]["COMPOUND_INCHIKEYBLOCK1"]
        
        if gnps_inchikeyblock1 != npatlas_inchikeyblock1 and gnps_inchikeyblock1 == "None":
            count += 1
            print(gnps_compound_name, gnps_compound["GNPSID"], gnps_inchikeyblock1, npatlas_inchikeyblock1, count)
            gnps_compound["COMPOUND_INCHIKEY"] = np_atlas_compound_by_name[0]["COMPOUND_INCHIKEY"]

filtered_gnps_with_npatlas_structures = [compound for compound in all_gnps_without_structure_fiter if len(compound["COMPOUND_INCHIKEY"]) > 10]
open("gnpslibraryjson_filteredv2.json", "w").write(json.dumps(filtered_gnps_with_npatlas_structures))

14543449